# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [18]:
import requests
import time
import re

from random import randint

In [19]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [20]:
from functools import wraps


def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} потребовалось {end - start:.6f} секунд')
        return result

    return wrapper

## Задание 2

In [21]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        print(f"Функция {func.__name__} была вызвана с параметрами:")
        print(f"Позиционные аргументы: {args}")
        print(f"Именованные аргументы: {kwargs}")
        return func(*args, **kwargs)

    return wrapper

## Задание 3

In [22]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        print(f"Функция {func.__name__} была вызвана {wrapper.calls} раз(а)")
        return func(*args, **kwargs)

    wrapper.calls = 0

    ...
    return wrapper

## Задание 4

In [23]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    cache = {}

    def fmemo(*args):
        if args in cache:
            print(f"Результат для {args} был взят из кэша")
            return cache[args]
        else:
            result = func(*args)
            cache[args] = result
            return result

    fmemo.cache = cache
    return fmemo

## Тестирование

In [24]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+', ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"


print(word_count('whole'))

Функция wrapper была вызвана 1 раз(а)
Функция word_count была вызвана с параметрами:
Позиционные аргументы: ('whole',)
Именованные аргументы: {}
word_count потребовалось 1.861709 секунд
Cлово whole встречается 176 раз


In [25]:
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [26]:
# измеряем время выполнения
print(fib(4))

fib потребовалось 0.000000 секунд
fib потребовалось 0.000001 секунд
fib потребовалось 0.000086 секунд
fib потребовалось 0.000000 секунд
fib потребовалось 0.000001 секунд
fib потребовалось 0.000000 секунд
fib потребовалось 0.000007 секунд
fib потребовалось 0.000014 секунд
fib потребовалось 0.000106 секунд
3


In [30]:
@memo
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [31]:
# измеряем время выполнения
print(fib(6))

fib потребовалось 0.000000 секунд
fib потребовалось 0.000000 секунд
fib потребовалось 0.000031 секунд
Результат для (1,) был взят из кэша
Результат для (2,) был взят из кэша
fib потребовалось 0.000007 секунд
fib потребовалось 0.000045 секунд
Результат для (3,) был взят из кэша
Результат для (4,) был взят из кэша
fib потребовалось 0.000005 секунд
fib потребовалось 0.000056 секунд
8
